# Idea: People tend to use bikes more on weekdays than weekends


# original 
##### Null Hypothesis: The proportion of people that bike during the weekends is more or the same to the proportion of people that bike on weekdays
### $h_0$ : $\dfrac{riders_{weekday}}{riders_{total}} \leqslant  \dfrac{riders_{weekend}}{riders_{total}}$

##### Alternative Hypothesis: The propportion of people that bike during the weekends is less than the proportion of people that bike on weekdays 
### $h_1$ : $\dfrac{riders_{weekday}}{riders_{total}} >\dfrac{riders_{weekend}}{riders_{total}}$

## I've modified my hypothesis to incorporate suggestions based from Sarah's comments (sj909)
### Null Hypothesis: The average amount of bike riders during the weekends is more or the same to the average amount of bike riders that bike on weekdays
### $h_0$ : $\mu_{weekday} \leqslant \mu_{weekend}$

### Alternative Hypothesis: The average amount of bike riders during the weekends is less than the average amount of bike riders that bike on weekdays
### $h_1$ : $\mu_{weekday} > \mu_{weekend}$

### I set signifiance level for $\alpha = 0.05$

Now i import package and other important helper functions.<br>
be advised i used the following packages that may not be in your default environment <br>
dateutil<br>
requests<br>
zipfile<br>
io

In [1]:
#/home/pui_user/PUI2018_msm796/HW4_msm796
#declare libraries
try:
    import urllib2 as urllib
except ImportError:
    import urllib.request as urllib
import datetime
from dateutil.relativedelta import relativedelta #DOWNLOAD THIS PACKAGE#
import os
import requests #DOWNLOAD THIS PACKAGE#
import zipfile #DOWNLOAD THIS PACKAGE#
import io #DOWNLOAD THIS PACKAGE#
import pandas as pd
%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import subprocess as sp
import os

#library#
    
#declare parameters
puidata = os.getenv("PUIDATA")
if puidata is None:
    os.environ["PUIDATA"] = "%s/PUIdata"%os.getenv("HOME")
    puidata = os.getenv("PUIDATA")

#declare helper functions
def download_citibike_data(yearstart,yearend,monthstart,monthend,output_path):
    '''
    change log:
    
    version 1.0 - Downloads citibike datasets zip files and unzips them. does checking if file already
    exists in path. currently has no input error handling logic.
    ---------------------------------------------------------------------------------------------
    Parameters:
    name - type - description
    yearstart - <int> - refers to the year where you want your download to start
    yearend - <int> - refers to the year where you want your download to start
    monthstart - <int> - refers to the month ssociated with yearstart where you want your download to start
    monthend - <int> - refers to the month associated with yearend where you want your download to start
    outputpath - <str> - refers to location where you want to dump the csv files
    
    example usage:
    download_citibike_data(2013,2018,6,8,puidata)
    - this downloads the citibike datasets from June 2013 to August 2018. Files will be unzipped at
    the puidata location.
    
    required libraries:
    datautil - required for year & month generation
    requests - for downloading the zip file
    zipfile - to unzip the file
    io - to read binary data and give it to zipfile; effectly keeping the entire process in memory
    '''
    #might include in error handling in the future for incorrect input
    curr_date = start_date=datetime.datetime(yearstart,monthstart,1).date()
    end_date=datetime.datetime(yearend,monthend,1).date()
    year_month=[start_date.strftime('%Y%m')]
    base_citi_url='https://s3.amazonaws.com/tripdata/'
    while curr_date < end_date:
        curr_date += relativedelta(months=1)
        year_month.append(curr_date.strftime('%Y%m'))
    for x in year_month:
    #need to change logic as csv filename convention isn't done properly#
    #will just base on dates found in directory#
        filename=x+"-citibike-tripdata.csv"
        if os.path.isfile(output_path+"/"+filename)==True:
            print("{} already exists!".format(filename))
        else:
            print("Downloading {}".format(filename))
            year=int(x[:4])
            if year>=2017:
                ending=".csv.zip"
            else:
                ending=".zip"
            url=base_citi_url+x+"-citibike-tripdata"+ending
            r = requests.get(url, stream=True)
            z = zipfile.ZipFile(io.BytesIO(r.content))
            z.extractall(output_path)
            
def append_all_citi_bike_data(input_path):
    '''
    developer's notes:
    please refactor me! thanks!
    change log:
    
    version 1.0 -currently has no input error handling logic or checking if file exists.

    Parameters:
    name - type - description
    
    input_path - <str> - location where the csv files are stored
    -Naively appends all files into one superfile
    
    required libraries:
    pandas - the output will be a dataframe, also used to load the files
    '''
    #currently implementing
    files_directory=os.listdir(input_path)
    csv_files = [x for x in files_directory if (".csv" in x) & (("citi" in x) | ("Citi" in x))]
    super_csv = pd.concat([pd.io.parsers.read_csv(input_path+"/"+f,engine='c',memory_map=True,low_memory='dtype') for f in csv_files],sort=False)
#    super_csv = pd.concat([pd.read_csv(input_path+"/"+f,engine='c',memory_map=True,low_memory='dtype') for f in csv_files])
    return super_csv

# I get all the 2016 data from citibike and then concatenate them into one dataframe

In [2]:
#downloads from beginning to latest available as of late September
download_citibike_data(2016,2016,1,12,puidata)
complete_set=append_all_citi_bike_data(puidata)

MemoryError: 

In [ ]:
#complete_set.to_csv(puidata+'/complete_bike_as_of_201808.csv')
#complete_set.to_csv(puidata+'/complete_2016_bike.csv')
#complete_set2=pd.read_csv(puidata+'/complete_2016_bike.csv',engine='c',memory_map=True,low_memory='dtype')

## Displaying the first five rows of the dataset

In [ ]:
complete_set.head()

## I generate the list of columns before i reduced the number of columns of the dataset

In [ ]:
complete_set.columns.get_values()

## Since the phenomenon i want to investigate only requires that information regarding days, i only keep starttime & stoptime

In [ ]:
drop_cols=[x for x in complete_set.columns.get_values() if x not in ['starttime','stoptime']]
complete_set2=complete_set.drop(drop_cols,axis=1)

## Then i display my first 10 rows

In [ ]:
complete_set2.head(10)

## But we're curious whether or not there are more users during weekdays vs weekends, relatively speaking, so let's derive some days from our remaining columns. So first we remove rows with missing entries for our datetime fields because we cannot use them for this investigation

In [ ]:
complete_set3 = complete_set2.dropna()

## Let's now derive our days

In [ ]:
complete_set3['day_start'] = complete_set3['starttime'].apply(lambda x : datetime.datetime.strptime(x, "%m/%d/%Y %H:%M:%S")).apply(lambda x: x.strftime('%A'))
complete_set3['day_end'] = complete_set3['stoptime'].apply(lambda x : datetime.datetime.strptime(x, "%m/%d/%Y %H:%M:%S")).apply(lambda x: x.strftime('%A'))
#complete_set3['day_start'] = complete_set3['starttime'].astype('datetime64[ns]').apply(lambda x: x.strftime('%A'))
#complete_set3['day_end'] = complete_set3['stoptime'].astype('datetime64[ns]').apply(lambda x: x.strftime('%A'))

## So now we have our days. But let's check if there are situations when some rides cross over midnight so that we can know if we can be indifferent of the variable of choice to describe our weekdays and weekends.


In [ ]:
complete_set3.head(5)

In [ ]:
complete_set3[complete_set3['day_start']!=complete_set3['day_end']].head(10)

## It appears we have a set of observations that start and end trip at different days. Since we're looking into how the day falls under a weekend or weekday affects ridership, then we should use the day when the ride begins as our basis.

In [ ]:
weekend=['Saturday','Sunday']
complete_set3['part_of_week']=complete_set3['day_start'].apply(lambda x: 'Weekend' if x in weekend else 'Weekday')

In [ ]:
complete_set3.head(5)

## Now let's plot our counts per day

In [ ]:
plt.figure(figsize=(30,10))
complete_set3.day_start.value_counts().plot(kind = 'bar',rot=0)
plt.xlabel("Day of week",fontsize=18)
plt.ylabel("Count",fontsize=18)
plt.show()

## Based on the bar graph, we get a general sense that weekdays will tend to have more rides than weekends.

# Distribution of weekdays

In [ ]:
plt.figure(figsize=(20,10))
complete_set3[complete_set3['part_of_week']=='Weekday'].groupby(['day_start'])['part_of_week'].count().plot(kind='bar',rot=0)
plt.xlabel("Day of week",fontsize=18)
plt.ylabel("Count",fontsize=18)
plt.show()

### Figure 2: We're dealing with a categorical distribution; a more generalized form of Bernoulli

# Distribution of weekends

In [ ]:
plt.figure(figsize=(20,10))
complete_set3[complete_set3['part_of_week']=='Weekend'].groupby(['day_start'])['part_of_week'].count().plot(kind='bar',rot=0)
plt.xlabel("Day of week",fontsize=18)
plt.ylabel("Count",fontsize=18)
plt.show()

### Figure 3: We're dealing with a Bernoulli distribution

# But really, we more interested with the Distribution by part of week

In [ ]:
plt.figure(figsize=(20,10))
complete_set3.groupby(['part_of_week'])['day_start'].count().plot(kind='bar',rot=0)
plt.xlabel("Day of week",fontsize=18)
plt.ylabel("Count",fontsize=18)
plt.show()

### Figure 3: We're dealing with a Bernoulli distribution
## But naturally, by comparing them with counts is inappropriate. You could come up with a misleading conclusion with the data. There are five days in a weekdays and two days in weekends. So let's get the average ridership during weekdays and weekends, then and also adjust them by normalizing them by their respective Standard Deviation to get a sense on comparing them if they were in the same scale.

In [ ]:
mean_wked = complete_set3[complete_set3['part_of_week']=='Weekend'].groupby(['day_start'])['part_of_week'].count().mean()
mean_wkdy = complete_set3[complete_set3['part_of_week']=='Weekday'].groupby(['day_start'])['part_of_week'].count().mean()
std_wked = complete_set3[complete_set3['part_of_week']=='Weekend'].groupby(['day_start'])['part_of_week'].count().std()
std_wkdy = complete_set3[complete_set3['part_of_week']=='Weekday'].groupby(['day_start'])['part_of_week'].count().std()

print("sample mean ridership for weekends: {:,} ,sample mean ridership for weekdays: {:,}".format(mean_wked,mean_wkdy))
print("Standard Error ridership for weekends: {} , Standard Error ridership for weekdays: {}".format(std_wked,std_wkdy))
print("the normalized sample mean ridership for weekends: {} ,the normalized sample mean ridership for weekdays: {}".format(mean_wked/std_wked,mean_wkdy/std_wkdy))